In [98]:
import sys
sys.path.insert(0, "D:\\Documents\\food_recipe_gen\\recipe_1m_analysis")
import os
import pandas as pd
import torch
import utils
import numpy as np

from torch.utils.data import Dataset, DataLoader, Sampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Data preprocessing

In [99]:
FOLDER_PATH = "D:\\Google Drive\\Catherning Folder\\THU\\Thesis\\Recipe datasets\\scirep-cuisines-detail"
FILE = "cleaned_data.pkl"

In [100]:
df=pd.read_pickle(os.path.join(FOLDER_PATH,FILE))
df=df.reset_index()
df["nb_ingr"]=df["ingredients"].apply(len)

In [101]:
vocab_ingrs = utils.Vocabulary()
for ingredients in df.loc[:,"ingredients"]:
    for ingr in ingredients:
        vocab_ingrs.add_word(ingr)
vocab_ingrs.word2idx

{'egg': 0,
 'yeast': 1,
 'wheat': 2,
 'milk': 3,
 'lard': 4,
 'pork': 5,
 'carrot': 6,
 'pea': 7,
 'onion': 8,
 'potato': 9,
 'maple_syrup': 10,
 'almond': 11,
 'honey': 12,
 'oat': 13,
 'date': 14,
 'vegetable_oil': 15,
 'whole_grain_wheat_flour': 16,
 'butter': 17,
 'lovage': 18,
 'clam': 19,
 'thyme': 20,
 'black_pepper': 21,
 'parsley': 22,
 'ginger': 23,
 'bay': 24,
 'celery': 25,
 'cinnamon': 26,
 'mustard': 27,
 'cane_molasses': 28,
 'raisin': 29,
 'cream': 30,
 'asparagus': 31,
 'olive_oil': 32,
 'pepper': 33,
 'garlic': 34,
 'tomato': 35,
 'cilantro': 36,
 'tea': 37,
 'jasmine': 38,
 'vegetable': 39,
 'brown_rice': 40,
 'lemon_juice': 41,
 'soy_sauce': 42,
 'white_wine': 43,
 'chicken': 44,
 'vanilla': 45,
 'rice': 46,
 'mushroom': 47,
 'chicken_broth': 48,
 'basil': 49,
 'porcini': 50,
 'mozzarella_cheese': 51,
 'tuna': 52,
 'lemon': 53,
 'beef': 54,
 'fish': 55,
 'cocoa': 56,
 'green_bell_pepper': 57,
 'oregano': 58,
 'rosemary': 59,
 'coffee': 60,
 'banana': 61,
 'squash': 

In [102]:
vocab_cuisine = utils.Vocabulary()
for cuisine in df.loc[:,"cuisine"]:
    vocab_cuisine.add_word(cuisine)

In [103]:
# # df["cuisine_id"]=vocab_cuisine.word2idx[df.loc["cuisine"]]
# class_vector=[]
# for el in df["cuisine"]:
#     class_vector.append(vocab_cuisine.word2idx[el])
# class_vector = torch.Tensor(class_vector)

## Data processing

In [104]:
MAX_INGR=max(df["nb_ingr"])
MAX_INGR

32

In [160]:
def ingr2idx(ingr_list):
    # If I didn't do the one-hot encoding by myself and used directly an embedding layer in the net, 
    # I would have to pad the input
    input_=[]
    for ingr in ingr_list:
        input_.append(vocab_ingrs.word2idx[ingr])
    input_ = torch.LongTensor(input_)
    onehot_enc = F.one_hot(input_.to(torch.int64), INPUT_SIZE)
    output = torch.sum(onehot_enc,0)
    return output

class RecipesDataset(Dataset):
    """Recipes dataset for cuisine classification. Only from ingredients for now"""

    def __init__(self, dataframe):
        """
        Args:
            file (string): Path to the file
        """
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        ingr = ingr2idx(self.data.loc[idx,"ingredients"])
        label = vocab_cuisine.word2idx[self.data.loc[idx,"cuisine"]]

        return ingr, label

In [161]:
# class StratifiedSampler(Sampler):
#     """Stratified Sampling
#     Provides equal representation of target classes in each batch
#     """
#     def __init__(self, class_vector, batch_size):
#         """
#         Arguments
#         ---------
#         class_vector : torch tensor
#             a vector of class labels
#         batch_size : integer
#             batch_size
#         """
#         self.n_splits = int(class_vector.size(0) / batch_size)
#         self.class_vector = class_vector

#     def gen_sample_array(self):
#         try:
#             from sklearn.model_selection import StratifiedShuffleSplit
#         except:
#             print('Need scikit-learn for this functionality')
        
#         s = StratifiedShuffleSplit(n_splits=self.n_splits, test_size=0.2)
#         X = th.randn(self.class_vector.size(0),2).numpy()
#         y = self.class_vector.numpy()
#         s.get_n_splits(X, y)

#         train_index, test_index = next(s.split(X, y))
#         return np.hstack([train_index, test_index])

#     def __iter__(self):
#         return iter(self.gen_sample_array())

#     def __len__(self):
#         return len(self.class_vector)

In [162]:
# def sampleFromClass(ds, k):
#     class_counts = {}
#     train_data = []
#     train_label = []
#     test_data = []
#     test_label = []
#     for data, label in ds:
#         c = label
#         class_counts[c] = class_counts.get(c, 0) + 1
#         if class_counts[c] <= k:
#             train_data.append(data)
#             train_label.append(label)
#         else:
#             test_data.append(data)
#             test_label.append(label)
#     train_data = torch.cat(train_data)
#     for ll in train_label:
#         print(ll)
#     train_label = torch.cat(train_label)
#     test_data = torch.cat(test_data)
#     test_label = torch.cat(test_label)

#     return (TensorDataset(train_data, train_label), 
#         TensorDataset(test_data, test_label))

In [172]:
INPUT_SIZE=len(vocab_ingrs)
EMBED_DIM1 = 300
EMBED_DIM2 = 64
NUM_CLASSES = len(vocab_cuisine) #51
BATCH_SIZE = 1
PRINT_FREQ = 2000

In [173]:
dataset = RecipesDataset(df[["ingredients","cuisine"]])
# sampler = StratifiedSampler(class_vector, BATCH_SIZE)
train_d,test_d = torch.utils.data.random_split(dataset, [len(dataset)-1000,1000])
# train_ds, test_ds = sampleFromClass(dataset, 5)
train_loader = DataLoader(train_d,batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(test_d, batch_size=1)

# Model

In [179]:
class Net(nn.Module):
    def __init__(self, vocab_size, embedding_dim1, embedding_dim2, num_classes):
        super(Net, self).__init__()
        self.layer_1 = nn.Linear(vocab_size, embedding_dim1, bias=True)
        self.layer_2 = nn.Linear(embedding_dim1, embedding_dim1, bias=True)
        self.layer_3 = nn.Linear(embedding_dim1, embedding_dim2, bias=True)
        self.output_layer = nn.Linear(embedding_dim2, num_classes, bias=True)

    def forward(self, x):
        out = F.relu(self.layer_1(x))
        out = F.relu(self.layer_2(out))
        out = F.relu(self.layer_3(out))
        out = self.output_layer(out)
        return out

net = Net(INPUT_SIZE, EMBED_DIM1, EMBED_DIM2, NUM_CLASSES)

## Training

In [180]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [181]:
for epoch in range(10):

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data[0]
        labels = data[1]
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % PRINT_FREQ == PRINT_FREQ-1:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 3.304
[1,  4000] loss: 2.862
[1,  6000] loss: 2.461
[1,  8000] loss: 2.339
[1, 10000] loss: 2.286
[1, 12000] loss: 2.204
[1, 14000] loss: 2.145
[1, 16000] loss: 2.077
[2,  2000] loss: 2.028
[2,  4000] loss: 2.019
[2,  6000] loss: 1.938
[2,  8000] loss: 1.956
[2, 10000] loss: 1.958
[2, 12000] loss: 1.867
[2, 14000] loss: 1.978
[2, 16000] loss: 1.925
[3,  2000] loss: 1.805
[3,  4000] loss: 1.803
[3,  6000] loss: 1.771
[3,  8000] loss: 1.811
[3, 10000] loss: 1.843
[3, 12000] loss: 1.787
[3, 14000] loss: 1.828
[3, 16000] loss: 1.771
[4,  2000] loss: 1.673
[4,  4000] loss: 1.735
[4,  6000] loss: 1.656
[4,  8000] loss: 1.749
[4, 10000] loss: 1.664
[4, 12000] loss: 1.627
[4, 14000] loss: 1.685
[4, 16000] loss: 1.691
[5,  2000] loss: 1.528
[5,  4000] loss: 1.603
[5,  6000] loss: 1.603
[5,  8000] loss: 1.555
[5, 10000] loss: 1.602
[5, 12000] loss: 1.612
[5, 14000] loss: 1.593
[5, 16000] loss: 1.594
[6,  2000] loss: 1.468
[6,  4000] loss: 1.513
[6,  6000] loss: 1.474
[6,  8000] 

## Testing

In [182]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs = data[0]
        labels = data[1]
        outputs = net(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test dataset: %d %%' % (
    100 * correct / total))


Accuracy of the network on the test dataset: 48 %
